In [1]:
!pip install tensorflow
!pip install nltk
!pip install colorama
!pip install flask

  Using cached protobuf-4.23.4-cp310-abi3-win_amd64.whl.metadata (540 bytes)
Using cached protobuf-4.23.4-cp310-abi3-win_amd64.whl (422 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.48.2 requires protobuf<4.0dev,>=3.12.0, but you have protobuf 4.23.4 which is incompatible.
layer-api 0.9.384528 requires protobuf<3.20,>=3.17.3, but you have protobuf 4.23.4 which is incompatible.


     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.5 MB 10.2 MB/s eta 0:00:01
   --------- ------------------------------ 0.4/1.5 MB 3.8 MB/s eta 0:00:01
   --------------- ------------------------ 0.6/1.5 MB 4.5 MB/s eta 0:00:01
   ----------------- ---------------------- 0.7/1.5 MB 3.4 MB/s eta 0:00:01
   ------------------- -------------------- 0.7/1.5 MB 3.3 MB/s eta 0:00:01
   --------------------- ------------------ 0.8/1.5 MB 3.3 MB/s eta 0:00:01
   ----------------------- ---------------- 0.9/1.5 MB 3.2 MB/s eta 0:00:01
   ------------------------ --------------- 0.9/1.5 MB 2.8 MB/s eta 0:00:01
   -------------------------- ------------- 1.0/1.5 MB 2.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.1/1.5 MB 2.7 MB/s eta 0:00:01
   ---------------

In [3]:
import json
import pickle
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Load the json file
with open('intents.json') as file:
    data = json.load(file)

training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

# Text preprocessing
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

# Model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(LSTM(128))  # LSTM layer with 128 internal units
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()

epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

# Save the trained model
model.save('chat-model')

# Save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 lstm (LSTM)                 (None, 128)               74240     
                                                                 
 dense (Dense)               (None, 16)                2064      
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 81)                1377      
                                                                 
Total params: 93953 (367.00 KB)
Trainable params: 93953 (367.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500


8/

INFO:tensorflow:Assets written to: chat-model\assets


In [5]:
import json
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import colorama
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open('intents.json') as file:
    data = json.load(file)

def chat():
    # Load trained model
    model = keras.models.load_model('chat-model')

    # Load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # Load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # Parameters
    max_len = 20

    while True:
        print(Fore.LIGHTBLUE_EX + 'User: ' + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == 'quit':
            print(Fore.GREEN + 'CurhatBot:' + Style.RESET_ALL, "Take care. See you soon.")
            break

        # Tokenize and pad input sequence
        sequence = tokenizer.texts_to_sequences([inp])
        padded_sequence = pad_sequences(sequence, truncating='post', maxlen=max_len)

        # Predict using the model
        result = model.predict(padded_sequence)
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + 'CurhatBot:' + Style.RESET_ALL, np.random.choice(i['responses']))

print(Fore.YELLOW + 'Start talking with CurhatBot, your Sharing AI Assistant. (Type quit to stop talking)' + Style.RESET_ALL)
chat()


Start talking with CurhatBot, your Sharing AI Assistant. (Type quit to stop talking)
1/1 [==============================] - 0s 316ms/step
CurhatBot: Hi there. How are you feeling today?
1/1 [==============================] - 0s 19ms/step
CurhatBot: Hi there. What brings you here today?
User: CurhatBot: Take care. See you soon.
